# Segmenting and Clustering Neighborhoods in Toronto, Canada

## Part : 1/3 Start

# Installing Libraries


In [21]:
#pip install beautifulsoup4 # for webscrapping
#pip install lxml # Python library which allows easy handling of XML and HTML files
#pip install requests # for handling request to websites
#print('Libraries Installed')

# Importing Libraries

In [96]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np

# Loading File

In [200]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(page,'html.parser')
#print(soup.prettify()) #Uncomment to view the html content

# Viewing Title of WebPage

In [134]:
match= soup.title.text
print(match)

List of postal codes of Canada: M - Wikipedia


# Viewing Required Data from WebPage

In [201]:
table_post = soup.find('table')
#print(table_post.prettify()) # Uncomment to view html content

# Extracting Data and Transforming into DataFrame

In [136]:
table_post = soup.find('table') # Extracting the required table
fields = table_post.find_all('td') # All required values are at 'td'

postcode = [] #Empty list to be appended
borough = [] #Empty list to be appended
neighbourhood = [] #Empty list to be appended

for i in range(0, len(fields),3): # Range Intitalizing from 0 to len(fields) which is no of 'td'=540, and step size of 3, since there are 3 columns 
    postcode.append(fields[i].text.strip()) # field 0, it is post code
    borough.append(fields[i+1].text.strip()) # filed 1, it is borough
    neighbourhood.append(fields[i+2].text.strip()) # field 2, it is neighborhood
        
df = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()# since all are list, we have to transform it to columns
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# DataFrame without Borough as Not Assigned

In [137]:
# only_boroughs = df[df['Borough'] == 'Not assigned'].reset_index() # This data will have Borough=Not Assigned, 77 entries
df_reqd_1 = df[df['Borough'] != 'Not assigned'].reset_index(drop=True) #`drop` parameter to avoid the old index being added as a column

df_reqd_1.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# Dimension of DataFrame

In [138]:
df_reqd_1.shape

(103, 3)

## Part : 1/3 End

## Part : 2/3 Start

# Loading GeoSpatial DataSet into DataFrame

In [139]:
geo_url = 'https://cocl.us/Geospatial_data' 
df_geo = pd.read_csv(geo_url)
df_geo.rename(columns={'Postal Code':'Postcode'}, inplace=True) #Renaming 'Postal Code' from GeoSpatial df to 'Postcode', same as df_reqd_1
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [140]:
df_reqd_2 = pd.merge(df_reqd_1,df_geo,on='Postcode')
df_reqd_2.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part : 2/3 End

## Part : 3/3 Start

# Libraries

In [141]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


# Get the Lat-Lon values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>toronto_explorer</em>

In [142]:
address = ': Toronto' # Borough which has toronto in it

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# Generating Map of Toronto

In [143]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_reqd_2['Latitude'], df_reqd_2['Longitude'], df_reqd_2['Borough'], df_reqd_2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Exploring Neighborhoods in Toronto

## Creating list which has Borough as Toronto

In [144]:
borough_names = list(df_reqd_2.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

## Create a new DataFrame with only boroughs that contain the word Toronto

In [145]:
df_reqd_3 = df_reqd_2[df_reqd_2['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(df_reqd_3.shape)
df_reqd_3.head()

(39, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


# Visualizing Neighborhoods in Toronto


In [146]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_reqd_3['Latitude'], df_reqd_3['Longitude'], df_reqd_3['Borough'], df_reqd_3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

# Loading Foursquare API for exploring location

In [147]:
CLIENT_ID = '4GRAFMPW30ENDUZ23OEXFIUVJSCLQWT1ZL5NW3AORHJIGF14' # your Foursquare ID
CLIENT_SECRET = 'PPJ5YWDGH2D0BHWFGUYXEEV04DNEHEDS5PLTZXCYMVVKZDA5'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' , CLIENT_ID)
print('CLIENT_SECRET:',CLIENT_SECRET)

Your credentails:
CLIENT_ID:  4GRAFMPW30ENDUZ23OEXFIUVJSCLQWT1ZL5NW3AORHJIGF14
CLIENT_SECRET: PPJ5YWDGH2D0BHWFGUYXEEV04DNEHEDS5PLTZXCYMVVKZDA5


# Now, let's get the top 100 venues that are in  within a radius of 500 meters.

## Defining a Function

In [175]:
LIMIT = 200 # limit of number of venues returned by Foursquare API
RADIUS = 2000 # Radius of interest
def getNearbyVenues(names, latitudes, longitudes, RADIUS=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes): # we can also use PostCode and Borough here if required
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            RADIUS, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Calling the Function

In [176]:
toronto_venues = getNearbyVenues(names=df_reqd_3['Neighbourhood'],
                                   latitudes=df_reqd_3['Latitude'],
                                   longitudes=df_reqd_3['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

# Converting the Info to DataFrame


In [177]:
print(toronto_venues.shape)
toronto_venues.head()

(3878, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
3,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
4,"Regent Park, Harbourfront",43.65426,-79.360636,Distillery Sunday Market,43.650075,-79.361832,Farmers Market


# Grouping Data and Checking the No of Venues in each Neighborhood

In [178]:
toronto_venues.groupby(['Neighborhood']).count() # we can also groupby Postcode and Borough, make changes in the function

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
Business reply mail Processing Centre,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",100,100,100,100,100,100
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100


# Unique Venue Category's Count

In [179]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 242 uniques categories.


# Analyzing Each Neighborhood

In [180]:

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
 
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(3878, 243)


,Neighbourhood,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Next, let's group rows by Neighborhood and by taking the mean of the frequency of occurrence of each category

In [181]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(39, 243)


,Neighbourhood,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Berczy Park,0.00,0.00,0.01,0.0,0.00,0.00,0.02,0.00,0.00,...,0.01,0.0,0.01,0.00,0.0,0.00,0.0,0.0,0.02,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.00,0.02,0.0,0.00,0.00,0.01,0.02,0.03,...,0.00,0.0,0.02,0.00,0.0,0.01,0.0,0.0,0.00,0.0
2,Business reply mail Processing Centre,0.00,0.00,0.03,0.0,0.01,0.00,0.00,0.01,0.01,...,0.00,0.0,0.00,0.01,0.0,0.00,0.0,0.0,0.00,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.01,0.01,0.01,0.0,0.00,0.02,0.01,0.00,0.00,...,0.01,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.02,0.0
4,Central Bay Street,0.00,0.00,0.01,0.0,0.00,0.00,0.01,0.02,0.00,...,0.00,0.0,0.02,0.00,0.0,0.00,0.0,0.0,0.03,0.0


# Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [182]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [183]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Plaza,Dessert Shop,Italian Restaurant,Park,Liquor Store,Neighborhood,Baseball Stadium,Brewery
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Park,Gift Shop,Bakery,Asian Restaurant,Athletics & Sports,Restaurant,Cocktail Bar,Supermarket
2,Business reply mail Processing Centre,Coffee Shop,Brewery,Park,Italian Restaurant,Café,Bakery,Beach,Indian Restaurant,Bar,BBQ Joint
3,"CN Tower, King and Spadina, Railway Lands, Har...",Park,Coffee Shop,Hotel,Italian Restaurant,Gym,Café,Spa,Movie Theater,Dessert Shop,Beer Bar
4,Central Bay Street,Coffee Shop,Café,Restaurant,Park,Gastropub,Yoga Studio,Japanese Restaurant,Pizza Place,Bubble Tea Shop,Supermarket


# Clustering

In [184]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 2, 0, 3, 4, 3, 0, 1, 1])


# Adding Cluster Label to Our DataFrame

In [188]:
# add clustering labels
#neighborhoods_venues_sorted.drop(['Cluster Label'],axis=1)
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

toronto_merged = df_reqd_3

# merge toronto_grouped with df_reqd_3 to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Restaurant,Café,Park,Japanese Restaurant,Plaza,Liquor Store,Bookstore,Theater,Hotel
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Café,Park,Italian Restaurant,Theater,Japanese Restaurant,Hotel,Pizza Place,Dance Studio,Beer Bar
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Coffee Shop,Japanese Restaurant,Italian Restaurant,Gastropub,Café,Thai Restaurant,Diner,Park,Theater,Plaza
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Japanese Restaurant,Café,Italian Restaurant,Thai Restaurant,Plaza,Park,Gastropub,Supermarket,Creperie
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Coffee Shop,Pub,Beach,Breakfast Spot,Bakery,BBQ Joint,Japanese Restaurant,Ice Cream Shop,Thai Restaurant,Sushi Restaurant


# Visualizing Clusters on Map

In [190]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# This Looks Fantastically Clustered

# Let us examine each cluster

### Examine each cluster and determine the discriminating venue categories that distinguish each cluster.

### Cluster 1

In [192]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Park,Japanese Restaurant,Plaza,Liquor Store,Bookstore,Theater,Hotel
3,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Café,Italian Restaurant,Thai Restaurant,Plaza,Park,Gastropub,Supermarket,Creperie
5,Downtown Toronto,0,Coffee Shop,Café,Plaza,Dessert Shop,Italian Restaurant,Park,Liquor Store,Neighborhood,Baseball Stadium,Brewery
8,Downtown Toronto,0,Coffee Shop,Café,Park,Plaza,Hotel,Sandwich Place,Yoga Studio,Vegetarian / Vegan Restaurant,Japanese Restaurant,Thai Restaurant
10,Downtown Toronto,0,Coffee Shop,Café,Park,Plaza,Yoga Studio,Brewery,Gym,Scenic Lookout,Hotel,Restaurant
13,Downtown Toronto,0,Coffee Shop,Café,Plaza,Sandwich Place,Yoga Studio,Hotel,Vegetarian / Vegan Restaurant,Park,Liquor Store,Thai Restaurant
16,Downtown Toronto,0,Coffee Shop,Café,Plaza,Italian Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio,Hotel,Park,Baseball Stadium,Theater
32,Downtown Toronto,0,Park,Coffee Shop,Hotel,Italian Restaurant,Gym,Café,Spa,Movie Theater,Dessert Shop,Beer Bar
34,Downtown Toronto,0,Coffee Shop,Café,Plaza,Dessert Shop,Park,Italian Restaurant,Art Gallery,Thai Restaurant,Yoga Studio,Liquor Store
36,Downtown Toronto,0,Coffee Shop,Café,Park,Plaza,Yoga Studio,Vegetarian / Vegan Restaurant,Sandwich Place,Hotel,Neighborhood,Sporting Goods Shop


### Cluster 2

In [193]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,1,Coffee Shop,Italian Restaurant,Café,Bakery,Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Deli / Bodega,Bookstore
20,Central Toronto,1,Coffee Shop,Italian Restaurant,Park,Bakery,Indian Restaurant,Café,Deli / Bodega,Restaurant,Pizza Place,Japanese Restaurant
21,Central Toronto,1,Italian Restaurant,Coffee Shop,Sushi Restaurant,Café,Park,Trail,Japanese Restaurant,Yoga Studio,Restaurant,Middle Eastern Restaurant
23,Central Toronto,1,Italian Restaurant,Bakery,Coffee Shop,Sushi Restaurant,Café,Japanese Restaurant,Fast Food Restaurant,Park,Bookstore,Burger Joint
26,Central Toronto,1,Italian Restaurant,Park,Café,Bakery,Restaurant,Sushi Restaurant,Coffee Shop,Indian Restaurant,Yoga Studio,Bookstore
28,West Toronto,1,Park,Bakery,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Pub,Grocery Store,Gas Station,Bar
29,Central Toronto,1,Italian Restaurant,Park,Café,Sushi Restaurant,Bakery,Restaurant,Dessert Shop,Indian Restaurant,French Restaurant,Coffee Shop
31,Central Toronto,1,Italian Restaurant,Café,Park,Sushi Restaurant,Spa,Hotel,Gym,Grocery Store,Ice Cream Shop,French Restaurant


### Cluster Label 3

In [197]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,2,Coffee Shop,Pub,Beach,Breakfast Spot,Bakery,BBQ Joint,Japanese Restaurant,Ice Cream Shop,Thai Restaurant,Sushi Restaurant
15,East Toronto,2,Park,Brewery,Beach,Coffee Shop,Café,Bar,Bakery,BBQ Joint,Indian Restaurant,Italian Restaurant
17,East Toronto,2,Coffee Shop,Park,Brewery,Café,French Restaurant,Bakery,Diner,Bar,Vietnamese Restaurant,Ice Cream Shop
38,East Toronto,2,Coffee Shop,Brewery,Park,Italian Restaurant,Café,Bakery,Beach,Indian Restaurant,Bar,BBQ Joint


### Cluster Label 4

In [198]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,3,Coffee Shop,Café,Park,Italian Restaurant,Theater,Japanese Restaurant,Hotel,Pizza Place,Dance Studio,Beer Bar
2,Downtown Toronto,3,Coffee Shop,Japanese Restaurant,Italian Restaurant,Gastropub,Café,Thai Restaurant,Diner,Park,Theater,Plaza
6,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Park,Gastropub,Yoga Studio,Japanese Restaurant,Pizza Place,Bubble Tea Shop,Supermarket
18,Central Toronto,3,Coffee Shop,Park,Italian Restaurant,Sushi Restaurant,Bakery,Pub,Sandwich Place,Sporting Goods Shop,Spa,Bus Line
24,Central Toronto,3,Café,Coffee Shop,Park,Grocery Store,Italian Restaurant,Dessert Shop,Ice Cream Shop,Spa,Restaurant,Sushi Restaurant
27,Downtown Toronto,3,Coffee Shop,Clothing Store,Ramen Restaurant,Dessert Shop,Japanese Restaurant,Beer Bar,Tapas Restaurant,Supermarket,Restaurant,Theater
33,Downtown Toronto,3,Italian Restaurant,Park,Coffee Shop,Café,Gastropub,Dance Studio,Spa,Restaurant,Juice Bar,Hotel
35,Downtown Toronto,3,Park,Coffee Shop,Café,Restaurant,Thai Restaurant,Gastropub,Bakery,Japanese Restaurant,Pub,Diner
37,Downtown Toronto,3,Coffee Shop,Japanese Restaurant,Gastropub,Park,Italian Restaurant,Café,Diner,Hotel,Theater,Grocery Store


### Cluster Label 5

In [199]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,4,Café,Italian Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Bar,Dessert Shop,Restaurant,Indian Restaurant,Park,Bakery
9,West Toronto,4,Café,Italian Restaurant,Park,Bar,Brewery,Coffee Shop,Cocktail Bar,Indian Restaurant,Ethiopian Restaurant,Restaurant
11,West Toronto,4,Café,Coffee Shop,Bakery,Pizza Place,Restaurant,Cocktail Bar,Asian Restaurant,Dessert Shop,Vegetarian / Vegan Restaurant,Italian Restaurant
12,East Toronto,4,Café,Greek Restaurant,Park,Bakery,Coffee Shop,Italian Restaurant,Pizza Place,Ice Cream Shop,Yoga Studio,Caribbean Restaurant
14,West Toronto,4,Café,Coffee Shop,Park,Gift Shop,Bakery,Asian Restaurant,Athletics & Sports,Restaurant,Cocktail Bar,Supermarket
22,West Toronto,4,Café,Italian Restaurant,Coffee Shop,Gastropub,Brewery,Bar,Restaurant,Bakery,Pizza Place,Flea Market
25,West Toronto,4,Coffee Shop,Café,Restaurant,Bakery,Park,Bar,Italian Restaurant,Gastropub,Eastern European Restaurant,Flower Shop
30,Downtown Toronto,4,Pizza Place,Coffee Shop,Park,Café,Dessert Shop,Bakery,Italian Restaurant,Vegetarian / Vegan Restaurant,Art Gallery,Yoga Studio


## Part 3/3 End

# Thank You